# Condo.ca Python scraping

In [1]:
from requests_html import HTMLSession
from bs4 import BeautifulSoup

In [2]:
#get all source code from url
s=HTMLSession()
url='https://condos.ca/toronto/condos-for-sale'
def getdata(url):
    r= s.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    return soup


# get all 3k condo urls

In [6]:
from itertools import chain
#get invidual condo url
def individual_condo_url(one_url):
    soup=getdata(one_url)
    div = soup.find_all('a',{'class':'styles___Link-sc-54qk44-1 cncVOT'})
    individual_condo_url=[]
    for i in div:
        individual_condo_url.append('https://condos.ca' + i['href'])
    return individual_condo_url

#get the last page number
url= 'https://condos.ca/toronto/condos-for-sale?page=1'
soup=getdata(url)
div = soup.find_all('ul',{'class':'styles___Pagination-sc-a7byxw-0 dxtxIX'})
pages = []
for i in div:
    pages.append(i.text)
last_page=int(pages[0][-2:])

#get all 3k+ condo urls in one list
all_pages = []
for page in range(1,last_page+1):
    url= 'https://condos.ca/toronto/condos-for-sale?page='+ str(page)
    all_pages.append(url)
all_condos_urls = []
for i in all_pages:
    c = individual_condo_url(i)
    all_condos_urls.append(c)
final_all_condos_urls = list(chain(*all_condos_urls)) #return all values from list
len(final_all_condos_urls)

3179

In [7]:
# write list into txt file
# textfile = open("a_file.txt", "w")
# for element in final_all_condos_urls:
#     textfile.write(element + "\n")
# textfile.close()

#read list from txt file
my_file = open('3179_urls.txt', "r")
  
# reading the file
data = my_file.read()
  
# replacing end splitting the text 
# when newline ('\n') is seen.
data_into_list = data.split("\n")
len(data_into_list)

3180

In [64]:
len(data_into_list[2300:])

880

In [69]:
third_part_url=data_into_list[2300:]

In [73]:
third_part_url[229]

'https://condos.ca/toronto/casa-3-50-charles-st-e/unit-5406-C5563497'

In [76]:
len(third_part_url)

878

# get 2 urls dynamic element

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time
from selenium.common.exceptions import WebDriverException,InvalidSessionIdException

# gdit_url_test=['https://condos.ca/toronto/19-lippincott-street-e-york/w5582863','https://condos.ca/toronto/7-on-the-park-1395-lawrence-ave-w-1461-lawrence-ave-w/unit-704-W5596686','https://www.sladkfj.com/','https://condos.ca/toronto/70-upper-canada-drive-townhouses-70-78-upper-canada-dr-2-44-oaken-gate-way/c5594293']
gdit_url_2300 = data_into_list[2300:] 
# gdit_url_rest = final_all_condos_urls[1500:] #the rest of urls

def selenium_dynamic(urls):

    global driver
    options = webdriver.ChromeOptions()
    options.add_argument('start-maximized')
    #options.add_experimental_option('excludeSwitches',['enable-automation'])
    options.add_experimental_option('detach',True) #will keep the browser open until python script completes
    # options.add_experimental_option('useAutomationExtension',False) hide those codes controlled by automated software

    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    soup_all=[]
    for url in urls:
        try:
            driver = webdriver.Chrome(options=options,service=Service(ChromeDriverManager().install()))
            driver.maximize_window()
            driver.implicitly_wait(30)
            driver.get(url)
            y = 1000
            for timer in range(0,50):
                    driver.execute_script("window.scrollTo(0, "+str(y)+")")
                    y += 300  
                    time.sleep(1)
                    if y >8000:     
                        break
            soup = BeautifulSoup(driver.page_source,'html.parser')
            driver.close()
        except (WebDriverException,InvalidSessionIdException) as e:
            driver.close()
            continue #skip the exception and continue the loop

            
        soup_all.append(soup)
    return soup_all

soup_all=selenium_dynamic(gdit_url_2300)



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/101.0.4951.41/chromedriver_mac64.zip
Driver has been saved in cache [/Users/xiaoxuchen/.wdm/drivers/chromedriver/mac64/101.0.4951.41]


====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [/Users/xiaoxuchen/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [/Users/xiaoxuchen/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [/Users/xiaoxuchen/.wd

In [10]:
len(soup_all)

878

In [11]:


# #second way
with open("output3.html", "w") as file:
    file.write(str(soup_all))





In [12]:
# from bs4 import BeautifulSoup

with open("output3.html") as fp:
    soup_1 = BeautifulSoup(fp, 'html.parser')
len(soup_1)

1757

# self-built function of dynamic element from each condo

In [13]:
#MLS number in each single condo page
def MLS_number_in_each_condo():
    all=[]
    for i in soup_all:
        div=i.find_all('div',{'class':"styles___BrokrageAndMLSCont-sc-n2657u-0 gtejC"})
        if div ==[]: #make those cannot find this class to be NA
            div = ['NA']
        else:
            div = div
        for i in range(0,len(div)):
            if div  == ['NA']:
                all.append('#:NA') #make those prices without class info to be 0
            else:
                all.append(div[i].text)
    MLS_number_in_each_condo = []
    for j in range(0,len(all)):
        MLS_number_in_each_condo.append(all[j].split('#:',1)[1].strip())
    return MLS_number_in_each_condo
len(MLS_number_in_each_condo())

878

In [14]:
#get bed,bath and parking info

from itertools import chain
import re

def bed_bath_parking(strings):
    all=[]
    for i in soup_all:
        div=i.find_all('div',{'class':"styles___Details-sc-ka5njm-10 emaPbE"})
        if div ==[]: #make those cannot find this class to be NA
            div = ['NA']
        else:
            div = div
        for i in range(0,len(div)):
            if div  == ['NA']:
                all.append('') #make those prices without class info to be ''
            else:
                all.append(div[i].text)
    test_list_new = []
    for i in all:
        if 'Studio' in i:
            i=i.replace("Studio",'Studio Bed') #change studio to studio bed
        elif i=='':
            i='0 Bed |0 Bath | 0 Parking' #insert info to empty element
        else:
            i=i
        test_list_new.append(i)


    total_1 = []
    for i in range(0,len(test_list_new)):
        total_1.append(test_list_new[i].replace('NA','0')) #replace all NA in string to 0
    total = []
    for i in range(0,len(total_1)) :
        if strings=='Bed':
            total.append(re.findall(r'(.*?) '+ strings,total_1[i])) 
        else:
            total.append(re.findall(r'(\d+) ' + strings,total_1[i]))
    return list(chain(*total)) 
len(bed_bath_parking('Parking'))

878

In [15]:
#get price

def condo_price():
    all=[]
    for i in soup_all:
        div=i.find_all('div',{'class':"styles___Price-sc-ka5njm-23 iRyAnp"})
        if div ==[]: #make those cannot find this class to be NA
            div = ['NA']
        else:
            div = div
        for i in range(0,len(div)):
            if div  == ['NA']:
                all.append('$0') #make those prices without class info to be 0
            else:
                all.append(div[i].text)
    total_1 = []
    for i in range(0,len(all)):
        total_1.append(all[i].replace(',',''))
    prices=[]
    for j in range(0,len(total_1)):
        prices.append(total_1[j].split('$')[1].strip())
    condo_price=[int(x) for x in prices]
    return condo_price
len(condo_price())

878

In [16]:
#sqft
def total_sqft():
    alls=[]
    for i in soup_all:
        div=i.find_all('div',{'class':"styles___BlockContainer-sc-1cv9cf1-1 eWVqJL styles___Container-sc-1cv9cf1-0 TitleValueBlockV2___Styled_Container-sc-xxjho6-0 QcERn styles___TitleValueBlock-sc-14y6qt-2 fEpIdS"})
        alls.append(div)

    second_lists = []
    for i in range(0,len(alls)):
        first_list=[] #put it here will make it separate each iteration result
        for tag in alls[i]:
            if ('Size' in tag.text.strip())==True:
                first_list.append(tag.text.strip())
            else:
                first_list.append('')  
        if list(chain(*first_list))==[]: #if all elements in list equal to ''
            first_list=['']
        else:
            first_list=list(filter(None, first_list))  #remove all '' elements in list
        second_lists+=first_list
    third_list=[]
    for i in second_lists:
        third_list.append(i.replace('Size',''))
    return third_list
len(total_sqft())


878

In [17]:
#price per sqft
import numpy as np
import pandas as pd

def price_per_sqft():

    all=[]
    for i in soup_all:
        div=i.find_all('div',{'class':"styles___BlockContainer-sc-1cv9cf1-1 eWVqJL styles___Container-sc-1cv9cf1-0 TitleValueBlockV2___Styled_Container-sc-xxjho6-0 QcERn styles___TitleValueBlock-sc-14y6qt-2 fEpIdS"})
        if div ==[]: #make those cannot find this class to be NA
            div = ['NA']
        else:
            div = div
        all.append(div)

    second_loop = []
    for i in range(0,len(all)):
        first_loop = []
        if all[i]==['NA']:
            first_loop.append('Price/sqft$0') #make those [] div to have Price tag
        else:
            [first_loop.append(tag.text.strip()) for tag in all[i] ]    
        second_loop.append(first_loop)
    # third_loop = []
    # for i in range(0,len(second_loop)): #hardcode to add price/sqft to index 1008
    #     if i==1008:
    #         second_loop[i].extend(['Price/sqft$0'])
    #     else:
    #         second_loop[i]=second_loop[i]
    #     third_loop.append(second_loop[i])
    final_list=list(chain(*second_loop))
    s1=pd.Series(final_list)

    a=s1.str.extract(r'(?<=Price/sqft)(.*$)') # get all values after specific words
    first_price_per_sqft=a.dropna().values.tolist() #drop all na values and change dataframe to list
    price_per_sqft=list(chain(*first_price_per_sqft)) #merge 2 lists into one and return all values
    return price_per_sqft
len(price_per_sqft())


878

In [18]:
# street
def street():
    all=[]
    for i in soup_all:
        div=i.find_all('h1',{'class':"styles___Title-sc-ka5njm-6 ccBSix"})
        if div ==[]: #make those cannot find this class to be NA
            div = ['NA']
        else:
            div = div
        for i in range(0,len(div)):
            if div  == ['NA']:
                all.append('') #make those street without class info to be 0
            else:
                all.append(div[i].text)
    return all
len(street())

878

In [19]:
# get maintenance fee
def maint_fee():
    alls=[]
    for i in soup_all:
        div=i.find_all('div',{'class':"styles___BlockContainer-sc-1cv9cf1-1 eWVqJL styles___Container-sc-1cv9cf1-0 TitleValueBlockV2___Styled_Container-sc-xxjho6-0 QcERn styles___TitleValueBlock-sc-14y6qt-2 fEpIdS"})
        if div ==[]: #make those cannot find this class to be NA
            div = ['NA']
        else:
            div = div
        alls.append(div)
        
    second_lists = []
    for i in range(0,len(alls)):  
        first_list=[] #put it here will make it separate each iteration result
        if alls[i]==['NA']:
            first_list.append('Maintenance fees$0') #make those [] div to have Price tag
        else:
            [first_list.append(tag.text.strip()) if ('Maintenance fees' in tag.text.strip())==True else first_list.append('') for tag in alls[i] ]
            
        if list(chain(*first_list))==[]: #if all elements in list equal to ''
            first_list=['']
        else:
            first_list=list(filter(None, first_list))  #remove all '' elements in list
        second_lists+=first_list #merge loop lists into one list

    final_lists =second_lists
    maint_fee=[]
    for i in final_lists:
        if i!='':
            i=i.replace('Maintenance fees','')
        else:
            i=''
        maint_fee.append(i)
    return maint_fee
len(maint_fee())

878

In [20]:
#get region before toronto
def region_toronto():
    address1 = []
    for i in soup_all:
        div=i.find_all('a',{'class':'styles___AddressInlineLink-sc-ka5njm-29 ewdunm','href':True})

        if div ==[]:
            div = ['NA']
        else:
            div = div

        for i in range(0,len(div)):
            if div  == ['NA']:
                address1.append('NA') #make those region without class info to be NA
            else:
                address1.append(div[i].text)
    return address1
len(region_toronto())

878

In [21]:
#age of building
def age_of_building():
    alls=[]
    for i in soup_all:
        div=i.find_all('div',{'class':"styles___BlockContainer-sc-1cv9cf1-1 eWVqJL styles___Container-sc-1cv9cf1-0 TitleValueBlockV2___Styled_Container-sc-xxjho6-0 QcERn styles___TitleValueBlock-sc-14y6qt-2 fEpIdS"})
        alls.append(div)

    second_lists = []
    for i in range(0,len(alls)):  
        first_list=[] #put it here will make it separate each iteration result
        for tag in alls[i]:
            if ('Age of building' in tag.text.strip())==True:
                first_list.append(tag.text.strip())
            else:
                first_list.append('')  
        if list(chain(*first_list))==[]: #if all elements in list equal to ''
            first_list=['']
        else:
            first_list=list(filter(None, first_list))  #remove all '' elements in list
        second_lists+=first_list #merge loop lists into one list

    final_lists =second_lists #if not, second_lists will be []
    age_of_building=[]
    for i in final_lists:
        if i!='':
            i=i.split('building',1)[1]
        else:
            i=''
        age_of_building.append(i)
    return age_of_building
len(age_of_building())


878

In [22]:
#household_composition and commute
def household_compo_vs_commute(strings):
    all=[]
    for i in soup_all:
        div=i.find_all('div',{'class':'styles___InfoHolder-sc-tub2qj-1 fWFzJo'})  
        if div ==[]:
            div = ['NA']
        else:
            div = div
        all.append(div)    
    second_lists = []
    for i in range(0,len(all)):  
        first_list=[] #put it here will make it separate each iteration result
        if all[i]==['NA']:
            append_text='0%'+strings
            first_list.append(append_text) #make those [] div to have Price tag
        else:
            [first_list.append(tag.text.strip()) if (strings in tag.text.strip())==True else first_list.append('') for tag in all[i] ]
            
        if list(chain(*first_list))==[]: #if all elements in list equal to ''
            first_list=['']
        else:
            first_list=list(filter(None, first_list))  #remove all '' elements in list
        second_lists+=first_list #merge loop lists into one list

    final_lists =second_lists
    single_family=[]
    for i in final_lists:
        if i!='':
            i=i.replace(strings,'')
        else:
            i=''
        single_family.append(i)
    return single_family
    
household_compo_singlefamily=household_compo_vs_commute('Single family')
household_compo_multifamily=household_compo_vs_commute('Multi family')
household_compo_singleperson=household_compo_vs_commute('Single person')
household_compo_multiperson=household_compo_vs_commute('Multi person')
commute_transit=household_compo_vs_commute('Transit')
commute_car=household_compo_vs_commute('Car')

len(household_compo_singlefamily)


878

In [23]:
#near_by info 
def near_by_info(strings):
    all=[]
    for i in soup_all:
        div=i.find_all('div',{'class':'styles___Flex-sc-1lfxfux-0 fFJWAv styles___PlaceInfo-sc-1r4w5sn-4 eHOZBs'})
        if div ==[]:
            div = ['NA']
        else:
            div = div
        all.append(div)    
    second_lists = []
    for i in range(0,len(all)):  
        first_list=[] #put it here will make it separate each iteration result
        if all[i]==['NA']:
            append_text='0'+strings
            first_list.append(append_text) #make those [] div to have Price tag
        else:
            [first_list.append(tag.text.strip()) if (strings in tag.text.strip())==True else first_list.append('') for tag in all[i] ]
            
        if list(chain(*first_list))==[]: #if all elements in list equal to ''
            first_list=['']
        else:
            first_list=list(filter(None, first_list))  #remove all '' elements in list
        second_lists+=first_list #merge loop lists into one list

    final_lists =second_lists
    single_family=[]
    for i in final_lists:
        if i!='':
            i=i.replace(strings,'')
        else:
            i=''
        single_family.append(i)
    return single_family
near_by_schools = near_by_info('Schools')
near_by_grocery_stores = near_by_info('Grocery stores')
near_by_restaurants = near_by_info('Bars & Restaurants')
len(near_by_schools)

878

In [24]:
#population
def population(index):
    all=[]
    for i in soup_all:
        div=i.find_all('div',{'class':'recharts-legend-wrapper'})
        if div ==[]:
            div = ['NA']
        else:
            div = div

        for i in range(0,len(div)):
            if div ==['NA']:
                all.append('0%(0 - 14 years old)0%(15 - 19 years old)0%(20 - 34 years old)0%(35 - 49 years old)0%(50 - 64 years old)0%(65+ years old)')
            else:
                if i%2==0:
                    all.append(div[i].text)
    a = []
    for i in all:
        a.append(re.sub(r"\s?\(.*?\)", r"", i)) #change all strings to nothing
    population_0_14_years =[] #make each iteration result append only one time
    for i in range(0,len(a)):   
        population_0_14_years.append(a[i].split('%')[index])
    return population_0_14_years
population_0_14_years = population(0)
population_15_19_years = population(1)
population_20_34_years = population(2)
population_35_49_years = population(3)
population_50_64_years = population(4)
population_65_and_more = population(5)
len(population_65_and_more)

878

In [25]:
#Tenancy & Property Type
def tenancy_vs_property_type(index):
    all=[]
    for i in soup_all:
        div=i.find_all('div',{'class':"sc-gsDJrp styles___NoMarginRow-sc-146e13k-1 styles___DetailKeyContainer-sc-1j5ezg1-0 deybmG gkHXVW bvBYTD"})
        if div ==[]:
            div = ['NA']
        else:
            div = div

        for i in range(0,len(div)):
            if div ==['NA']:
                all.append('Owned0%Rented0%Detached0%Semi Detached0%Duplex0%Low-Rise Apartment0%High-Rise Apartment0%',)
            else:
                all.append(div[i].text)
    a = []
    for i in all:
        a.append(re.findall(r"(\d+)%", i)) #extract only numbers before %
    tenancy_property_type_owned=[]
    for i in range(0,len(a)):
        tenancy_property_type_owned.append(a[i][index])
    return tenancy_property_type_owned
tenancy_property_type_owned = tenancy_vs_property_type(0)
tenancy_property_type_rented = tenancy_vs_property_type(1)
tenancy_property_type_detached = tenancy_vs_property_type(2)
tenancy_property_type_semi_detached = tenancy_vs_property_type(3)
tenancy_property_type_duplex = tenancy_vs_property_type(4)
tenancy_property_type_low_rise = tenancy_vs_property_type(5)
tenancy_property_type_high_rise = tenancy_vs_property_type(6)
len(tenancy_property_type_duplex)

878

In [26]:
#avg income per household or individual
def avg_income_per_year(index):
    all=[]
    for i in soup_all:
        div=i.find_all('div',{'class':"sc-gsDJrp styles___NoMarginRow-sc-146e13k-1 styles___NoMarginRow-sc-7urk12-2 deybmG gkHXVW oA-dSi"})
        if div ==[]:
            div = ['NA']
        else:
            div = div

        for i in range(0,len(div)):
            if div ==['NA']:
                all.extend(('1','1','Household:$0,0 /yrIndividual:$0,0 /yr'))
            else:
                all.append(div[i].text)

    a = []
    for i in range(2,len(all),3):
        a.append(all[i].replace(',','')) #change comma to nothing
    b =[]
    for j in range(0,len(a)):
        b.append(re.findall(r"(\d+)",a[j])) #extract only numbers from strings
    avg_income_household_yr = []
    for income in range(0,len(b)):
        avg_income_household_yr.append(b[income][index])
    return avg_income_household_yr
avg_income_household_yr=avg_income_per_year(0)
avg_income_individual_yr = avg_income_per_year(1)
len(avg_income_household_yr)

878

In [27]:
# education 
def education_info(index):
    all=[]
    for i in soup_all:
        div=i.find_all('div',{'class':"sc-gsDJrp styles___NoMarginRow-sc-146e13k-1 styles___NoMarginRow-sc-7urk12-2 deybmG gkHXVW oA-dSi"})

        if div ==[]:
            div = ['NA']
        else:
            div = div

        for i in range(0,len(div)):
            if div ==['NA']:
                all.extend(('1','No Diploma:0%High school:0%Trade certificate:0%College certificate:0%University certificate:0%Bachelor degree:0%Post graduate degree:0%','1'))
            else:
                all.append(div[i].text)
    edu_all = []
    for j in range(1,len(all),3):
        edu_all.append(all[j])

    new_test = []   #get new list after added trade certificate:0%
    for i in edu_all:
        if 'Trade certificate' not in i:
            i=re.sub(r'(\w\d%)(College)',r'\1Trade Certificate:0%\2',i) #add those missing trade certificate
        else:
            i=i
        new_test.append(i)
    
    b =[]
    for j in range(0,len(new_test)):
        b.append(re.findall(r"(\d+)",new_test[j])) #get all numbers from string

    new_b = [] #for the rest of missing, add all missing edu levels to 0%
    for j in b:
        if len(j)==5:
            j.insert(4,'0')
            j.insert(6,'0')
        elif len(j)==6:
            j.insert(6,'0')
        elif len(j)==4:
            j.extend(('0','0','0'))
        elif len(j)==3:
            j.extend(('0','0','0','0'))
        elif len(j)==2:
            j.extend(('0','0','0','0','0'))
        elif len(j)==1:
            j.extend(('0','0','0','0','0','0'))
        else:
            j=j
        new_b.append(j)

    edu_number = []  #add some education level into one
    for edu in range(0,len(new_b)):
        if index in (0,1,2):
            edu_number.append(new_b[edu][index])
        else:
            edu_number.append(str(int(new_b[edu][index])+int(new_b[edu][index+1])))
    return edu_number

edu_no_diploma = education_info(0)
edu_high_school = education_info(1)
edu_trade_cert = education_info(2)
edu_coll_or_univ_cert = education_info(3)
edu_ba_or_msc_cert = education_info(5)
len(edu_ba_or_msc_cert)


878

In [28]:
#language
import pandas as pd
import itertools

all=[]
for i in soup_all:
    div=i.find_all('div',{'class':"sc-gsDJrp styles___NoMarginRow-sc-146e13k-1 styles___NoMarginRow-sc-7urk12-2 deybmG gkHXVW oA-dSi"})

    if div ==[]:
        div = ['NA']
    else:
        div = div

    for i in range(0,len(div)):
        if div ==['NA']:
            all.extend(('English:0%Mandarin:0%Cantonese:0%French:0%Persian (Farsi):0%Russian:0%Spanish:0%Korean:0%Arabic:0%Other:0%','1','1'))
        else:
            all.append(div[i].text)

a= []
for i in range(0,len(all),3):
        a.append(all[i])
series = pd.Series(a)

# for people speak English
one = series.str.extract(r'(?<=' + 'English' + r':)(\d+)' +r'(?=%|percent\b)')
c = one.values.tolist()  #change from series to list
speak_English = list(chain(*c)) #extract all values from list

# for people speak European language
df=[]   
for i in ('Portuguese','Italian','Spanish','French','German','Polish','Russian','Ukrainian','Greek','Croatian','Romanian'):
    one=series.str.extract(r'(?<=' + i + r':)(\d+)' +r'(?=%|percent\b)')
    df.append(one)
df=pd.concat(df,axis=1) #append dataframes into one dataframe
df=df.fillna(0).astype('int')
sum_all=df.sum(axis=1,skipna=True)
speak_Europe_language = sum_all.values.tolist()

# for people from Africa
df=[]   
for i in ('Somali','Yoruba'):
    one=series.str.extract(r'(?<=' + i + r':)(\d+)' +r'(?=%|percent\b)')
    df.append(one)
df=pd.concat(df,axis=1) #append dataframes into one dataframe
df=df.fillna(0).astype('int')
sum_all=df.sum(axis=1,skipna=True)
speak_Africa_language = sum_all.values.tolist()

# for people from Middle East
df=[]   
for i in ('Hebrew','Arabic','Persian','Armenian','Farsi'):
    if i =='Farsi':
        one = series.str.extract(r'(?<='+ i +r'\)\:)(\d+)'+r'(?=%|percent\b)')
    else:
        one=series.str.extract(r'(?<=' + i + r':)(\d+)' +r'(?=%|percent\b)')
    df.append(one)
df=pd.concat(df,axis=1) #append dataframes into one dataframe
df=df.fillna(0).astype('int')
sum_all=df.sum(axis=1,skipna=True)
speak_middleEast_language = sum_all.values.tolist()

# for people from Asian
df=[]   
for i in ('Filipino','Mandarin','Cantonese','Vietnamese','Japanese','Korean','Tamil','Bengali','Urdu','Ilocano','Cebuano','Gujarati','Hindi','Panjabi'):
    if i in ('Filipino','Panjabi'):
        one = series.str.extract(r'(?<='+ i +r'\)\:)(\d+)'+r'(?=%|percent\b)')
    else:
        one=series.str.extract(r'(?<=' + i + r':)(\d+)' +r'(?=%|percent\b)')
    df.append(one)
df=pd.concat(df,axis=1) #append dataframes into one dataframe
df=df.fillna(0).astype('int')
sum_all=df.sum(axis=1,skipna=True)
speak_Asian_language = sum_all.values.tolist()

len(speak_English)

878

In [29]:
#brokerages
def brokerages():
    all=[]
    for i in soup_all:
        div=i.find_all('div',{'class':"styles___BrokrageAndMLSCont-sc-n2657u-0 gtejC"})
        if div ==[]: #make those cannot find this class to be NA
            div = ['NA']
        else:
            div = div
        for i in range(0,len(div)):
            if div  == ['NA']:
                all.append('') #make those prices without class info to be 0
            else:
                all.append(div[i].text)
    a = []
    for j in range(0,len(all)):
        a.append(all[j].split('BROKERAGE')[0])

    brokerages = []
    for broker in a:
        brokerages.append(broker.replace('Broker:','').strip())
    return brokerages
len(brokerages())

878

# create dataframe

In [80]:
df_condo_info_third_part = pd.DataFrame({
'MLS_number_condo': MLS_number_in_each_condo(), 
'condo_price':condo_price(),
'maint_fee':maint_fee(),
'avg_income_household_yr':avg_income_household_yr,
'avg_income_individual_yr':avg_income_individual_yr,
'total_sqft':total_sqft(),
'price_per_sqft':price_per_sqft(),
'street':street(),
'toronto_region':region_toronto(),
'bedrooms':bed_bath_parking('Bed'),
'bath':bed_bath_parking('Bath'),
'parking':bed_bath_parking('Parking'),
'age_of_building':age_of_building(),
'commute_transit': household_compo_vs_commute('Transit'),
'commute_car':household_compo_vs_commute('Car'),
'near_by_schools': near_by_info('Schools'),
'near_by_grocery_stores':near_by_info('Grocery'),
'near_by_bars&restaurants':near_by_info('Bars'),
'speak_English': speak_English,
'speak_Europe_language':speak_Europe_language,
'speak_Asian_language':speak_Asian_language,
'speak_Africa_language':speak_Africa_language,
'speak_middleEast_language':speak_middleEast_language,
'edu_no_diploma':edu_no_diploma,
'edu_high_school':edu_high_school,
'edu_trade_cert':edu_trade_cert,
'edu_coll_or_univ_cert':edu_coll_or_univ_cert,
'edu_ba_or_msc_cert':edu_ba_or_msc_cert,
'population_0_14_years': population_0_14_years,
'population_15_19_years':population_15_19_years,
'population_20_34_years':population_20_34_years,
'population_35_49_years':population_35_49_years,
'population_50_64_years':population_50_64_years,
'population_65_and_more':population_65_and_more,
'tenancy_property_type_owned': tenancy_property_type_owned,
'tenancy_property_type_rented':tenancy_property_type_rented,
'tenancy_property_type_detached':tenancy_property_type_detached,
'tenancy_property_type_semi_detached':tenancy_property_type_semi_detached,
'tenancy_property_type_duplex':tenancy_property_type_duplex,
'tenancy_property_type_low_rise':tenancy_property_type_low_rise,
'tenancy_property_type_high_rise':tenancy_property_type_high_rise,
'household_compo_singlefamily': household_compo_singlefamily,
'household_compo_multifamily':household_compo_multifamily,
'household_compo_singleperson':household_compo_singleperson,
'household_compo_multiperson':household_compo_multiperson,
'brokerages':brokerages(),
'single_condo_url':third_part_url
})  

df_condo_info_third_part


,MLS_number_condo,condo_price,maint_fee,avg_income_household_yr,avg_income_individual_yr,total_sqft,price_per_sqft,street,toronto_region,bedrooms,...,tenancy_property_type_semi_detached,tenancy_property_type_duplex,tenancy_property_type_low_rise,tenancy_property_type_high_rise,household_compo_singlefamily,household_compo_multifamily,household_compo_singleperson,household_compo_multiperson,brokerages,single_condo_url
0,C5571008,0,$477.25,67850,29821,,"$1,068",109 - 52 Forest Manor Rd,North York,0,...,0,0,0,92,61%,2%,27%,10%,"ROYAL LEPAGE CONNECT REALTY,",https://condos.ca/toronto/colours-of-emerald-c...
1,C5570163,749000,$486.75,80835,34868,,"$1,198",N2710 - 6 Sonic Way,North York,1+1,...,25,0,32,27,68%,6%,20%,6%,"RE/MAX REAL ESTATE CENTRE INC.,",https://condos.ca/toronto/sonic-1185-eglinton-...
2,C5573396,888000,$578.00,85849,46737,,"$1,198",1001 - 7 Kenaston Gdns,North York,2,...,0,0,2,91,50%,0%,46%,4%,"RE/MAX REALTRON REALTY INC.,",https://condos.ca/toronto/lotus-condominiums-7...
3,C5570253,950000,$591.98,67464,29159,,"$1,082",307 - 38 Forest Manor Rd,North York,3,...,0,0,0,93,63%,2%,26%,9%,"NU STREAM REALTY (TORONTO) INC.,",https://condos.ca/toronto/the-point-at-emerald...
4,C5571251,2350000,,103651,45035,3500 - 5000 sqft,$470 - $671,275 David Dunlap Circ,North York,4,...,4,1,31,27,63%,2%,32%,3%,"ROYAL LEPAGE SIGNATURE REALTY,",https://condos.ca/toronto/275-david-dunlap-cir...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873,C5089803,5688000,"$5,062.64",466099,171692,,"$1,408",309 - 1 Post Rd,North York,3,...,0,0,7,0,77%,4%,16%,3%,"HARVEY KALLES REAL ESTATE LTD.,",https://condos.ca/toronto/one-post-road-1-post...
874,C4499434,46000,$0.00,112927,55024,NA,NA,35 Merton St,Midtown | Central,0,...,0,0,3,94,36%,0%,58%,6%,"RE/MAX REALTRON REALTY INC.,",https://condos.ca/toronto/the-rio-35-merton-st...
875,_c218,460000,$0.00,65566,34264,0 - 499 sqft,NA,401 - 15 Neighbourhood Lane,Etobicoke,1,...,0,0,80,13,55%,0%,39%,5%,"Property.ca Inc., BrokerageMLS®#: _c218",https://condos.ca/toronto/the-king-s-mill-cond...
876,_c211,835000,NA,122024,62877,600 - 699 sqft,"$1,195 - $1,392",S563 - 475 Front Street East,Downtown,2,...,1,1,10,82,37%,0%,57%,6%,"Property.ca Inc., BrokerageMLS®#: _c211",https://condos.ca/toronto/canary-commons-475-f...


## show all columns 

In [81]:
import pandas as pd
from IPython.display import display
pd.set_option('display.max_columns', None)
display(df_condo_info_third_part)

,MLS_number_condo,condo_price,maint_fee,avg_income_household_yr,avg_income_individual_yr,total_sqft,price_per_sqft,street,toronto_region,bedrooms,bath,parking,age_of_building,commute_transit,commute_car,near_by_schools,near_by_grocery_stores,near_by_bars&restaurants,speak_English,speak_Europe_language,speak_Asian_language,speak_Africa_language,speak_middleEast_language,edu_no_diploma,edu_high_school,edu_trade_cert,edu_coll_or_univ_cert,edu_ba_or_msc_cert,population_0_14_years,population_15_19_years,population_20_34_years,population_35_49_years,population_50_64_years,population_65_and_more,tenancy_property_type_owned,tenancy_property_type_rented,tenancy_property_type_detached,tenancy_property_type_semi_detached,tenancy_property_type_duplex,tenancy_property_type_low_rise,tenancy_property_type_high_rise,household_compo_singlefamily,household_compo_multifamily,household_compo_singleperson,household_compo_multiperson,brokerages,single_condo_url
0,C5571008,0,$477.25,67850,29821,,"$1,068",109 - 52 Forest Manor Rd,North York,0,0,0,4 years old,47%,45%,0,1 stores,50 & Restaurants,24,0,38,0,12,10,21,5,21,43,16,5,36,21,14,8,28,72,5,0,0,0,92,61%,2%,27%,10%,"ROYAL LEPAGE CONNECT REALTY,",https://condos.ca/toronto/colours-of-emerald-c...
1,C5570163,749000,$486.75,80835,34868,,"$1,198",N2710 - 6 Sonic Way,North York,1+1,1,1,1 years old,38%,57%,0,2 stores,36 & Restaurants,46,7,15,0,5,15,23,2,24,36,19,7,23,24,19,9,44,54,0,25,0,32,27,68%,6%,20%,6%,"RE/MAX REAL ESTATE CENTRE INC.,",https://condos.ca/toronto/sonic-1185-eglinton-...
2,C5573396,888000,$578.00,85849,46737,,"$1,198",1001 - 7 Kenaston Gdns,North York,2,2,1,3 years old,39%,56%,2,3 stores,30 & Restaurants,26,6,37,0,16,6,16,3,21,54,9,3,30,25,16,18,62,37,2,0,0,2,91,50%,0%,46%,4%,"RE/MAX REALTRON REALTY INC.,",https://condos.ca/toronto/lotus-condominiums-7...
3,C5570253,950000,$591.98,67464,29159,,"$1,082",307 - 38 Forest Manor Rd,North York,3,2,1,-,47%,45%,0,1 stores,50 & Restaurants,27,0,22,0,19,12,21,5,20,41,16,5,34,21,15,9,39,61,2,0,0,0,93,63%,2%,26%,9%,"NU STREAM REALTY (TORONTO) INC.,",https://condos.ca/toronto/the-point-at-emerald...
4,C5571251,2350000,,103651,45035,3500 - 5000 sqft,$470 - $671,275 David Dunlap Circ,North York,4,5,3,-,26%,70%,0,5 stores,52 & Restaurants,49,2,20,0,2,9,28,4,27,31,15,6,18,25,23,12,45,55,16,4,1,31,27,63%,2%,32%,3%,"ROYAL LEPAGE SIGNATURE REALTY,",https://condos.ca/toronto/275-david-dunlap-cir...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873,C5089803,5688000,"$5,062.64",466099,171692,,"$1,408",309 - 1 Post Rd,North York,3,4,3,20 years old,14%,77%,2,0 stores,0 & Restaurants,54,2,24,0,7,10,17,0,17,56,12,9,20,12,27,20,97,2,92,0,0,7,0,77%,4%,16%,3%,"HARVEY KALLES REAL ESTATE LTD.,",https://condos.ca/toronto/one-post-road-1-post...
874,C4499434,46000,$0.00,112927,55024,NA,NA,35 Merton St,Midtown | Central,0,0,0,19 years old,58%,31%,2,6 stores,92 & Restaurants,68,8,2,0,2,7,16,3,20,54,8,2,33,24,17,16,29,71,1,0,0,3,94,36%,0%,58%,6%,"RE/MAX REALTRON REALTY INC.,",https://condos.ca/toronto/the-rio-35-merton-st...
875,_c218,460000,$0.00,65566,34264,0 - 499 sqft,NA,401 - 15 Neighbourhood Lane,Etobicoke,1,1,1,New,38%,55%,1,1 stores,9 & Restaurants,45,34,0,0,0,18,22,8,24,24,15,4,24,25,21,10,6,91,6,0,0,80,13,55%,0%,39%,5%,"Property.ca Inc., BrokerageMLS®#: _c218",https://condos.ca/toronto/the-king-s-mill-cond...
876,_c211,835000,NA,122024,62877,600 - 699 sqft,"$1,195 - $1,392",S563 - 475 Front Street East,Downtown,2,2,1,New,34%,30%,0,1 stores,99 & Restaurants,69,9,5,0,0,4,12,3,22,58,7,1,41,28,15,8,59,41,0,1,1,10,82,37%,0%,57%,6%,"Property.ca Inc., BrokerageMLS®#: _c211",https://condos.ca/toronto/canary-commons-475-f...


# write first part into csv file

In [82]:
# import os  
# os.makedirs('Python_scrap', exist_ok=True)  
# df_condo_info_first_part.to_csv('Python_scrap/condo_firstpart.csv')  

In [83]:
import os  
os.makedirs('Python_scrap', exist_ok=True)  
df_condo_info_third_part.to_csv('Python_scrap/df_condo_info_third_part.csv')  

# read csv file from local 

In [42]:
import pandas as pd
df_firstpart=pd.read_csv('df_condo_info_first_part.csv',index_col=[0])
df_firstpart

,MLS_number_condo,condo_price,maint_fee,avg_income_household_yr,avg_income_individual_yr,total_sqft,price_per_sqft,street,toronto_region,bedrooms,...,tenancy_property_type_low_rise,tenancy_property_type_high_rise,household_compo_singlefamily,household_compo_multifamily,household_compo_singleperson,household_compo_multiperson,brokerages,single_condo_url,latitude,longitude
0,C5608721,980000,$647.31,125204,64764,700 - 799 sqft,"$1,227 - $1,400",4708 - 125 Blue Jays Way,Downtown,2,...,1,98,29%,0%,59%,12%,"AIMHOME NEW TIMES REALTY,",https://condos.ca/toronto/king-blue-115-125-bl...,43.645941,-79.392054
1,C5608680,699800,$657.88,67850,29821,NaN,$813,1103 - 62 Forest Manor Rd,North York,2,...,0,92,61%,2%,27%,10%,"RIGHT AT HOME REALTY,",https://condos.ca/toronto/dream-tower-at-emera...,43.773944,-79.345246
2,C5608587,960000,$832.53,142425,63489,NaN,"$1,061",2008 - 35 Balmuto St,Downtown,2+1,...,3,96,33%,0%,54%,12%,"ROYAL LEPAGE GOLDEN RIDGE REALTY,",https://condos.ca/toronto/uptown-residences-35...,43.669472,-79.387488
3,C5608686,698800,$344.65,108578,60024,NaN,"$1,099",1111 - 111 Bathurst St,Downtown,1,...,15,76,35%,1%,56%,9%,"RE/MAX HALLMARK REALTY LTD.,",https://condos.ca/toronto/oneeleven-condos-111...,43.645108,-79.402790
4,W5608672,389900,$569.67,62102,27414,800 - 899 sqft,$434 - $487,2502 330 Dixon Rd,Etobicoke,2,...,0,90,66%,4%,23%,7%,"RE/MAX WEST REALTY INC.,",https://condos.ca/toronto/320-dixon-condos-320...,43.695785,-79.553155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,W5593847,849999,$649.74,137681,71016,NaN,"$1,059",1207 - 10 Park Lawn Rd,Etobicoke,2,...,2,94,42%,0%,53%,5%,"ROYAL LEPAGE CREDIT VALLEY REAL ESTATE,",https://condos.ca/toronto/westlake-encore-10-p...,43.623221,-79.483934
1496,W5593216,738000,$475.30,155193,78644,NaN,$971,104 - 59 Annie Craig Dr,Etobicoke,1+1,...,0,99,43%,0%,53%,3%,"HOMELIFE LANDMARK REALTY INC.,",https://condos.ca/toronto/ocean-club-waterfron...,43.624949,-79.478244
1497,C5590954,699000,$424.60,93644,49550,NaN,"$1,013",418 - 150 Sudbury St,West End,2,...,10,77,35%,0%,49%,16%,"RE/MAX REALTY SPECIALISTS INC.,",https://condos.ca/toronto/westside-gallery-lof...,43.641476,-79.424127
1498,C5589889,799900,$748.86,98845,51273,NaN,$947,509 - 15 Bruyeres Mews,Downtown,2+1,...,1,98,36%,0%,51%,13%,"ROYAL HERITAGE REALTY LTD.,",https://condos.ca/toronto/ltd-15-bruyeres-mews...,43.637196,-79.400689


In [36]:
import pandas as pd
df_secondpart=pd.read_csv('df_condo_info_second_part.csv',index_col=[0])
df_secondpart

,MLS_number_condo,condo_price,maint_fee,avg_income_household_yr,avg_income_individual_yr,total_sqft,price_per_sqft,street,toronto_region,bedrooms,...,tenancy_property_type_low_rise,tenancy_property_type_high_rise,household_compo_singlefamily,household_compo_multifamily,household_compo_singleperson,household_compo_multiperson,brokerages,single_condo_url,latitude,longitude
0,C5589784,830000,$507.02,89097,49317,NaN,"$1,358",3010 - 219 Fort York Blvd,Downtown,1+1,...,1,98,35%,0%,53%,12%,"EXP REALTY,",https://condos.ca/toronto/aquarius-at-waterpar...,43.637016,-79.405577
1,C5590127,899000,$442.68,107134,61408,NaN,"$1,373",3113 - 300 Front St W,Downtown,1+1,...,1,99,31%,0%,56%,13%,"RE/MAX HALLMARK REALTY LTD.,",https://condos.ca/toronto/300-front-street-wes...,43.644062,-79.389439
2,W5589736,738000,$567.52,95337,49056,NaN,$997,1210 - 2 Fieldway Rd,Etobicoke,1,...,1,80,49%,1%,45%,5%,"RE/MAX HALLMARK REALTY LTD.,",https://condos.ca/toronto/network-lofts-2-fiel...,43.643208,-79.527041
3,W5591026,295000,$735.00,49276,21895,800 - 899 sqft,$328 - $369,618 - 4673 Jane St,North York,2,...,1,76,65%,4%,25%,6%,"ARCREALTY INC.,",https://condos.ca/toronto/4745-jane-street-con...,43.770446,-79.520103
4,W5589991,828000,$737.36,121492,53956,NaN,"$1,127",1813 - 155 Legion Rd N,Etobicoke,2,...,19,44,53%,2%,39%,6%,"CENTURY 21 HERITAGE GROUP LTD.,",https://condos.ca/toronto/ilofts-155-legion-rd...,43.623942,-79.488447
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,W5570384,599000,$416.74,64941,26761,600 - 699 sqft,$857 - $998,1102 - 2520 Eglinton Ave,York Crosstown,1+1,...,30,24,62%,5%,29%,5%,"ROYAL LEPAGE FLOWER CITY REALTY,",https://condos.ca/toronto/2520-eglinton-avenue...,43.691409,-79.470807
796,E5571198,1250000,"$1,006.15",89979,38274,1200 - 1399 sqft,"$893 - $1,042",803 - 630 Greenwood Ave,East End,2+1,...,46,14,53%,2%,38%,6%,"POPE REAL ESTATE LIMITED,",https://condos.ca/toronto/platform-condos-1177...,43.681126,-79.332216
797,C5570965,3299000,"$2,132.60",240147,97719,1800 - 1999 sqft,"$1,650 - $1,833",804 - 3 Mcalpine St,Downtown,3,...,4,88,38%,0%,57%,4%,"SOTHEBY`S INTERNATIONAL REALTY CANADA,",https://condos.ca/toronto/domus-3-mcalpine-st-...,43.674032,-79.390473
798,C5571177,929900,$773.01,104943,54608,NaN,"$1,126",1602 - 170 Fort York Blvd,Downtown,2,...,2,97,35%,0%,52%,13%,"SOTHEBY`S INTERNATIONAL REALTY CANADA,",https://condos.ca/toronto/library-district-170...,43.639394,-79.399786


In [30]:
import pandas as pd
df_thirdpart=pd.read_csv('df_condo_info_third_part.csv',index_col=[0])
df_thirdpart

,MLS_number_condo,condo_price,maint_fee,avg_income_household_yr,avg_income_individual_yr,total_sqft,price_per_sqft,street,toronto_region,bedrooms,...,tenancy_property_type_low_rise,tenancy_property_type_high_rise,household_compo_singlefamily,household_compo_multifamily,household_compo_singleperson,household_compo_multiperson,brokerages,single_condo_url,latitude,longitude
0,C5571008,0,$477.25,67850,29821,NaN,"$1,068",109 - 52 Forest Manor Rd,North York,0,...,0,92,61%,2%,27%,10%,"ROYAL LEPAGE CONNECT REALTY,",https://condos.ca/toronto/colours-of-emerald-c...,43.773207,-79.345795
1,C5570163,749000,$486.75,80835,34868,NaN,"$1,198",N2710 - 6 Sonic Way,North York,1+1,...,32,27,68%,6%,20%,6%,"RE/MAX REAL ESTATE CENTRE INC.,",https://condos.ca/toronto/sonic-1185-eglinton-...,43.719765,-79.336312
2,C5573396,888000,$578.00,85849,46737,NaN,"$1,198",1001 - 7 Kenaston Gdns,North York,2,...,2,91,50%,0%,46%,4%,"RE/MAX REALTRON REALTY INC.,",https://condos.ca/toronto/lotus-condominiums-7...,43.766740,-79.385703
3,C5570253,950000,$591.98,67464,29159,NaN,"$1,082",307 - 38 Forest Manor Rd,North York,3,...,0,93,63%,2%,26%,9%,"NU STREAM REALTY (TORONTO) INC.,",https://condos.ca/toronto/the-point-at-emerald...,43.772208,-79.345351
4,C5571251,2350000,NaN,103651,45035,3500 - 5000 sqft,$470 - $671,275 David Dunlap Circ,North York,4,...,31,27,63%,2%,32%,3%,"ROYAL LEPAGE SIGNATURE REALTY,",https://condos.ca/toronto/275-david-dunlap-cir...,43.729762,-79.334672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873,C5089803,5688000,"$5,062.64",466099,171692,NaN,"$1,408",309 - 1 Post Rd,North York,3,...,7,0,77%,4%,16%,3%,"HARVEY KALLES REAL ESTATE LTD.,",https://condos.ca/toronto/one-post-road-1-post...,43.736369,-79.380535
874,C4499434,46000,$0.00,112927,55024,NaN,NaN,35 Merton St,Midtown | Central,0,...,3,94,36%,0%,58%,6%,"RE/MAX REALTRON REALTY INC.,",https://condos.ca/toronto/the-rio-35-merton-st...,43.696201,-79.394814
875,_c218,460000,$0.00,65566,34264,0 - 499 sqft,NaN,401 - 15 Neighbourhood Lane,Etobicoke,1,...,80,13,55%,0%,39%,5%,"Property.ca Inc., BrokerageMLS®#: _c218",https://condos.ca/toronto/the-king-s-mill-cond...,43.638436,-79.488235
876,_c211,835000,NaN,122024,62877,600 - 699 sqft,"$1,195 - $1,392",S563 - 475 Front Street East,Downtown,2,...,10,82,37%,0%,57%,6%,"Property.ca Inc., BrokerageMLS®#: _c211",https://condos.ca/toronto/canary-commons-475-f...,43.653004,-79.355554


In [43]:
df=pd.concat([df_firstpart, df_secondpart,df_thirdpart], ignore_index=True, sort=False)
df

,MLS_number_condo,condo_price,maint_fee,avg_income_household_yr,avg_income_individual_yr,total_sqft,price_per_sqft,street,toronto_region,bedrooms,...,tenancy_property_type_low_rise,tenancy_property_type_high_rise,household_compo_singlefamily,household_compo_multifamily,household_compo_singleperson,household_compo_multiperson,brokerages,single_condo_url,latitude,longitude
0,C5608721,980000,$647.31,125204,64764,700 - 799 sqft,"$1,227 - $1,400",4708 - 125 Blue Jays Way,Downtown,2,...,1,98,29%,0%,59%,12%,"AIMHOME NEW TIMES REALTY,",https://condos.ca/toronto/king-blue-115-125-bl...,43.645941,-79.392054
1,C5608680,699800,$657.88,67850,29821,NaN,$813,1103 - 62 Forest Manor Rd,North York,2,...,0,92,61%,2%,27%,10%,"RIGHT AT HOME REALTY,",https://condos.ca/toronto/dream-tower-at-emera...,43.773944,-79.345246
2,C5608587,960000,$832.53,142425,63489,NaN,"$1,061",2008 - 35 Balmuto St,Downtown,2+1,...,3,96,33%,0%,54%,12%,"ROYAL LEPAGE GOLDEN RIDGE REALTY,",https://condos.ca/toronto/uptown-residences-35...,43.669472,-79.387488
3,C5608686,698800,$344.65,108578,60024,NaN,"$1,099",1111 - 111 Bathurst St,Downtown,1,...,15,76,35%,1%,56%,9%,"RE/MAX HALLMARK REALTY LTD.,",https://condos.ca/toronto/oneeleven-condos-111...,43.645108,-79.402790
4,W5608672,389900,$569.67,62102,27414,800 - 899 sqft,$434 - $487,2502 330 Dixon Rd,Etobicoke,2,...,0,90,66%,4%,23%,7%,"RE/MAX WEST REALTY INC.,",https://condos.ca/toronto/320-dixon-condos-320...,43.695785,-79.553155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3155,C5089803,5688000,"$5,062.64",466099,171692,NaN,"$1,408",309 - 1 Post Rd,North York,3,...,7,0,77%,4%,16%,3%,"HARVEY KALLES REAL ESTATE LTD.,",https://condos.ca/toronto/one-post-road-1-post...,43.736369,-79.380535
3156,C4499434,46000,$0.00,112927,55024,NaN,NaN,35 Merton St,Midtown | Central,0,...,3,94,36%,0%,58%,6%,"RE/MAX REALTRON REALTY INC.,",https://condos.ca/toronto/the-rio-35-merton-st...,43.696201,-79.394814
3157,_c218,460000,$0.00,65566,34264,0 - 499 sqft,NaN,401 - 15 Neighbourhood Lane,Etobicoke,1,...,80,13,55%,0%,39%,5%,"Property.ca Inc., BrokerageMLS®#: _c218",https://condos.ca/toronto/the-king-s-mill-cond...,43.638436,-79.488235
3158,_c211,835000,NaN,122024,62877,600 - 699 sqft,"$1,195 - $1,392",S563 - 475 Front Street East,Downtown,2,...,10,82,37%,0%,57%,6%,"Property.ca Inc., BrokerageMLS®#: _c211",https://condos.ca/toronto/canary-commons-475-f...,43.653004,-79.355554


In [44]:
import os  
os.makedirs('Python_scrap', exist_ok=True)  
df.to_csv('final_condo_data.csv')  

# add latitude and longitude

In [37]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(open("output1.html", encoding="utf8"), "html.parser")
geo=[geo['content'] for geo in soup.find_all('meta',attrs={'tag':'meta',"name":'geo.position'})]
ids=[id.text.split('#:',1)[1].strip() for id in soup.find_all('div',{'class':"styles___BrokrageAndMLSCont-sc-n2657u-0 gtejC"}) ]
for j in zip(ids,geo):
    id,lat = j
    print(f"{id}:{lat}")